In [1]:
#Imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from PIL import Image
import clip
import os


In [2]:
torch.cuda.is_available()

True

In [22]:
data = pd.read_csv('cat_dog1.csv')
data

,Unnamed: 0,image,labels
0,0,dog.624.jpg,1
1,1,cat.170.jpg,0
2,2,dog.419.jpg,1
3,3,dog.19.jpg,1
4,4,cat.227.jpg,0
...,...,...,...
1995,1995,dog.481.jpg,1
1996,1996,dog.159.jpg,1
1997,1997,cat.576.jpg,0
1998,1998,dog.190.jpg,1


In [33]:
#a-Spliting the dataset into training and testing sets with 20% for test dataset
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [34]:
train_data.sort_values(train_data.columns[2], axis = 0, inplace=True) #sort by label
train_cats = train_data[train_data['labels'] == 0]



In [5]:
# Load the CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"

model, transform = clip.load("ViT-B/32", device) 

d:\Mariam\Python\Python312\Lib\site-packages\clip\clip.py:57: UserWarning: C:\Users\Family/.cache/clip\ViT-B-32.pt exists, but the SHA256 checksum does not match; re-downloading the file
  warnings.warn(f"{download_target} exists, but the SHA256 checksum does not match; re-downloading the file")
100%|███████████████████████████████████████| 338M/338M [02:33<00:00, 2.31MiB/s]


In [35]:
preproc_img_train_cat = []
preproc_img_train_dog = []
preproc_img_test = []

In [36]:
for file in train_cats['image']:
  preproc_img_train_cat.append(transform(Image.open('cat_dog1/' + file )).unsqueeze(0).to(device))
for file in train_dogs['image']:
  preproc_img_train_dog.append(transform(Image.open('cat_dog1/' + file )).unsqueeze(0).to(device))

for file in test_data['image']:
  preproc_img_test.append(transform(Image.open('cat_dog1/' + file)).unsqueeze(0).to(device))

In [37]:
#encode the inputs
image_embeddings_train_dog = []
image_embeddings_train_cat = []
image_embeddings_test = []
torch.cuda.empty_cache()
with torch.no_grad():
  for image in preproc_img_train_cat:
    image_embeddings_train_cat.append((model.encode_image(image)).cpu().numpy())
  
  for image in preproc_img_train_dog:
    image_embeddings_train_dog.append((model.encode_image(image)).cpu().numpy())

  for image in preproc_img_test:
    image_embeddings_test.append((model.encode_image(image)).cpu().numpy())


In [48]:
image_embeddings_train_cat = np.vstack(image_embeddings_train_cat)
image_embeddings_train_dog = np.vstack(image_embeddings_train_dog)
image_embeddings_test = np.vstack(image_embeddings_test)


array([[ 0.2212  ,  0.0993  , -0.099   , ...,  0.9565  ,  0.007534,
         0.122   ],
       [-0.3103  , -0.1823  ,  0.3867  , ...,  0.7407  ,  0.1398  ,
         0.3574  ],
       [-0.07806 , -0.2463  ,  0.06335 , ...,  1.275   ,  0.2473  ,
        -0.02036 ],
       ...,
       [ 0.2091  ,  0.2673  , -0.241   , ...,  0.4856  , -0.001846,
         0.248   ],
       [-0.3276  ,  0.1192  ,  0.31    , ...,  1.1875  , -0.01619 ,
         0.277   ],
       [ 0.03693 , -0.09326 ,  0.0961  , ...,  0.6284  , -0.2384  ,
         0.1028  ]], dtype=float16)

In [53]:
#Fisher's Algorithm
c = 0.1
means = []
means.append(image_embeddings_train_cat.mean(axis = 0))
means.append(image_embeddings_train_dog.mean(axis = 0))
S =  np.cov(image_embeddings_train_cat, rowvar=False) + np.cov(image_embeddings_train_dog, rowvar=False)
S_inv = np.linalg.inv(S)
mean_dif = means[1] - means[0]
w = c*np.dot(S_inv, mean_dif)

In [55]:
#predictions with c = 0.1
predict_c = np.dot(image_embeddings_test, w.T)
print(predict_c)

[-4.66712454e+00  5.25411342e-01 -2.78909745e+00  6.90284201e+01
 -1.50379462e+01 -4.74721040e+00 -7.05946066e+00 -6.48999716e+00
  4.07027964e+00 -7.08755957e+00 -1.64724517e+01  7.06060236e+01
 -2.35394283e+00 -8.38942538e+00 -7.14627341e+00 -1.15357510e+01
  6.31637300e+01 -2.72438807e+00  7.12043848e+01 -4.62374878e+00
  6.81812881e+00  6.95621988e+01  6.88343617e+01  6.31478792e+01
  6.29491432e+01 -4.34518146e+00 -8.70594084e-02  7.53102546e+01
 -9.76096886e+00  5.73699155e+01  6.64739703e+01  6.86950400e+01
  8.22530153e+01 -2.37774168e+00  7.14381174e+01 -7.13097463e-01
 -1.26009656e+00  7.88376590e+01  7.67098556e+01 -1.24749695e+01
  7.48026886e+01  8.31474917e+00  8.65483204e+01  6.62445429e+01
  3.66240177e-01  6.87379335e+01 -8.93111721e+00  5.44339385e+01
  6.53176939e+01 -8.08842858e+00  6.82138472e+01  8.32365707e+01
 -6.28279435e+00 -5.74284231e+00 -1.71683887e+01  1.83831234e+00
 -5.07656861e+00  5.17775415e+01 -9.25022978e+00  7.58007233e+01
 -8.68641757e+00  8.25965

In [ ]:
#Scores

In [ ]:
#Bonus

C = [0.2, 0.3, 0.4, 0.5, 0.7]
W = []
for c in C:
  W.append(c*np.dot(S_inv, mean_dif))



In [ ]:
predict_C = []
for w in W:
  predict_C.append(np.dot(image_embeddings_test, w.T))

In [ ]:
#Scores
